In [1]:
import urllib.request, urllib.error, urllib.parse
import requests
import csv
from urllib.request import build_opener, HTTPCookieProcessor, Request

#PUT JOURNAL URL HERE
url = 'https://onlinelibrary.wiley.com/loi/14676486'


opener = build_opener(HTTPCookieProcessor())
request = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
response = opener.open(request, timeout=30)
allContent = str(response.read())

In [2]:
def toSheetHeader(name):
    # field names  
    fields = ['Title', 'Abstract','Journal']  
    
    # name of csv file  
    filename = name + " SCRAPED.csv"
    
    # writing to csv file  
    with open(filename, 'w') as csvfile:  
        # creating a csv writer object  
        csvwriter = csv.writer(csvfile)  
        
        # writing the fields  
        csvwriter.writerow(fields)  

In [3]:
def toSheet(rows,name):
    # name of csv file  
    filename = name + " SCRAPED.csv"
    
    # writing to csv file  
    with open(filename, 'a') as csvfile:  
        # creating a csv writer object  
        csvwriter = csv.writer(csvfile)  
        
        # writing the data rows  
        csvwriter.writerows(rows) 

In [4]:
allarticles = []


#GETTING JOURNAL TITLE


journal_index = allContent.find('<title')
journal_index = allContent.find('>',journal_index) + 1
journal = ""
while(allContent[journal_index]!=":"):
    journal = journal + allContent[journal_index]
    journal_index = journal_index+1


journal = journal.replace("\\n"," ")

toSheetHeader(journal)
                    
#FINDING THE LINK TO THE ISSUE, allContent is all of the code for the original URL                    
issue_index = 0

while(allContent.find('loi-tab-item',issue_index,len(allContent))!=-1 and 
      allContent.find('2015',allContent.find('loi-tab-item',issue_index,len(allContent)),len(allContent))!=-1):
    
    issue_index = allContent.find('loi-tab-item',issue_index,len(allContent))
    i = issue_index-100
    i = allContent.find('href=', i, len(allContent)) + 6
    
    issue_link = ""
                                  
    while(allContent[i]!='"'):
        issue_link = issue_link+allContent[i]
        i = i+1
    
    #getting code for ISSUE LINK here
    opener = build_opener(HTTPCookieProcessor())
    request = Request("https://onlinelibrary.wiley.com"+issue_link, headers={'User-Agent': 'Mozilla/5.0'})
    response = opener.open(request, timeout=30)
    web1Content = str(response.read())

    
    #finding issue
    curr_index = 0
    while(web1Content.find('"loi__issue"',curr_index,len(web1Content))!=-1):
        curr_index = web1Content.find('"loi__issue"',curr_index,len(web1Content))
        
        iss_link_index = web1Content.find("href=",curr_index,len(web1Content))
        iss_link = ""
        i = iss_link_index+6
        while(web1Content[i]!='"'):
            iss_link = iss_link+web1Content[i]
            i = i+1
           
        
        opener = build_opener(HTTPCookieProcessor())
        request = Request("https://onlinelibrary.wiley.com"+iss_link, headers={'User-Agent': 'Mozilla/5.0'})
        response = opener.open(request, timeout=30)
        webContent = str(response.read())
    
    
        article_index = 0  
        ind = 0
        while(webContent.find('issue-item__title visitable',article_index,len(webContent))!=-1):
            currarticle = []
            article_index = webContent.find('issue-item__title visitable',article_index,len(webContent))
        
            link_index = webContent.find("href=",article_index,len(webContent))
            link = ""
            
            i = link_index+6
            while(webContent[i]!='"'):
                link = link+webContent[i]
                i = i+1

            title = ""
      
            i = webContent.find("<h2",i,len(webContent))
            i = webContent.find(">",i,len(webContent))+1
            
            while(webContent[i]!='<'):
                title = title+webContent[i]
                i = i+1

            title = title.replace("\\n","")
            currarticle.append(title)
            #print("\033[1m",title,"\033[0;0m")

        
      
            #OPENING LINK TO ARTICLE
            opener = build_opener(HTTPCookieProcessor())
            request = Request("https://onlinelibrary.wiley.com"+link, headers={'User-Agent': 'Mozilla/5.0'})
            response = opener.open(request, timeout=30)
            articleContent = str(response.read())
            print("https://onlinelibrary.wiley.com"+link)
        
            execute = True
            abstract_index = articleContent.find('main abstractlang_en main')
           
            if (abstract_index == -1):
                abstract_index = articleContent.find('article-section__content')
            if (abstract_index == -1):
                abstract_index = articleContent.find('<div class="section1')
            if (abstract_index == -1):
                execute = False
            print(abstract_index)
            
            if(execute):    
                abstract_index = articleContent.find("<p",abstract_index,len(articleContent))
                abstract_index = articleContent.find(">",abstract_index,len(articleContent))
                i = abstract_index+1
      
                end_abstract_index = articleContent.find("</p>",abstract_index,len(articleContent))

                abstract = ""

                while(i!=end_abstract_index):
                    abstract = abstract+articleContent[i]
                    i = i + 1
    
                abstract = abstract.replace("\\n"," ")
                currarticle.append(abstract)

            currarticle.append(journal)
            allarticles.append(currarticle)
            toSheet(allarticles,journal)
            allarticles=[]
            article_index = article_index+1
            issue_index = issue_index+1
            curr_index = curr_index +1


https://onlinelibrary.wiley.com/doi/10.1111/joms.12497
-1
https://onlinelibrary.wiley.com/doi/10.1111/joms.12518
53464
https://onlinelibrary.wiley.com/doi/10.1111/joms.12522
56538
https://onlinelibrary.wiley.com/doi/10.1111/joms.12524
53335
https://onlinelibrary.wiley.com/doi/10.1111/joms.12520
54612
https://onlinelibrary.wiley.com/doi/10.1111/joms.12533
56794
https://onlinelibrary.wiley.com/doi/10.1111/joms.12543
55269
https://onlinelibrary.wiley.com/doi/10.1111/joms.12608
52443
https://onlinelibrary.wiley.com/doi/10.1111/joms.12581
54586
https://onlinelibrary.wiley.com/doi/10.1111/joms.12582
54858
https://onlinelibrary.wiley.com/doi/10.1111/joms.12521
-1
https://onlinelibrary.wiley.com/doi/10.1111/joms.12496
-1
https://onlinelibrary.wiley.com/doi/10.1111/joms.12489
-1
https://onlinelibrary.wiley.com/doi/10.1111/joms.12605
62038
https://onlinelibrary.wiley.com/doi/10.1111/joms.12563
62099
https://onlinelibrary.wiley.com/doi/10.1111/joms.12573
52065
https://onlinelibrary.wiley.com/doi/

https://onlinelibrary.wiley.com/doi/10.1111/joms.12407
59221
https://onlinelibrary.wiley.com/doi/10.1111/joms.12335
55473
https://onlinelibrary.wiley.com/doi/10.1111/joms.12393
58055
https://onlinelibrary.wiley.com/doi/10.1111/joms.12419
-1
https://onlinelibrary.wiley.com/doi/10.1111/joms.12373
-1
https://onlinelibrary.wiley.com/doi/10.1111/joms.12388
-1
https://onlinelibrary.wiley.com/doi/10.1111/joms.12534
56795
https://onlinelibrary.wiley.com/doi/10.1111/joms.12289
52175
https://onlinelibrary.wiley.com/doi/10.1111/joms.12527
54986
https://onlinelibrary.wiley.com/doi/10.1111/joms.12531
60214
https://onlinelibrary.wiley.com/doi/10.1111/joms.12430
52362
https://onlinelibrary.wiley.com/doi/10.1111/joms.12432
60371
https://onlinelibrary.wiley.com/doi/10.1111/joms.12528
56765
https://onlinelibrary.wiley.com/doi/10.1111/joms.12529
53954
https://onlinelibrary.wiley.com/doi/10.1111/joms.12443
59291
https://onlinelibrary.wiley.com/doi/10.1111/joms.12449
-1
https://onlinelibrary.wiley.com/doi/

https://onlinelibrary.wiley.com/doi/10.1111/joms.12296
-1
https://onlinelibrary.wiley.com/doi/10.1111/joms.12295
-1
https://onlinelibrary.wiley.com/doi/10.1111/joms.12282
54958
https://onlinelibrary.wiley.com/doi/10.1111/joms.12217
56071
https://onlinelibrary.wiley.com/doi/10.1111/joms.12288
56249
https://onlinelibrary.wiley.com/doi/10.1111/joms.12255
53355
https://onlinelibrary.wiley.com/doi/10.1111/joms.12251
53520
https://onlinelibrary.wiley.com/doi/10.1111/joms.12334
-1
https://onlinelibrary.wiley.com/doi/10.1111/joms.12294
-1
https://onlinelibrary.wiley.com/doi/10.1111/joms.12293
-1
https://onlinelibrary.wiley.com/doi/10.1111/joms.12316
58724
https://onlinelibrary.wiley.com/doi/10.1111/joms.12279
57789
https://onlinelibrary.wiley.com/doi/10.1111/joms.12318
55419
https://onlinelibrary.wiley.com/doi/10.1111/joms.12320
55097
https://onlinelibrary.wiley.com/doi/10.1111/joms.12317
53558
https://onlinelibrary.wiley.com/doi/10.1111/joms.12315
53381
https://onlinelibrary.wiley.com/doi/10.

https://onlinelibrary.wiley.com/doi/10.1111/joms.12191
-1
https://onlinelibrary.wiley.com/doi/10.1111/joms.12157
-1
https://onlinelibrary.wiley.com/doi/10.1111/joms.12172
54896
https://onlinelibrary.wiley.com/doi/10.1111/joms.12178
51428
https://onlinelibrary.wiley.com/doi/10.1111/joms.12124
53275
https://onlinelibrary.wiley.com/doi/10.1111/joms.12129
51285
https://onlinelibrary.wiley.com/doi/10.1111/joms.12122
55997
https://onlinelibrary.wiley.com/doi/10.1111/joms.12156
-1
https://onlinelibrary.wiley.com/doi/10.1111/joms.12139
54014
https://onlinelibrary.wiley.com/doi/10.1111/joms.12165
55667
https://onlinelibrary.wiley.com/doi/10.1111/joms.12148
54876
https://onlinelibrary.wiley.com/doi/10.1111/joms.12154
52540
https://onlinelibrary.wiley.com/doi/10.1111/joms.12151
55688
https://onlinelibrary.wiley.com/doi/10.1111/joms.12147
53970
https://onlinelibrary.wiley.com/doi/10.1111/joms.12146
54454
https://onlinelibrary.wiley.com/doi/10.1111/joms.12141
52614
https://onlinelibrary.wiley.com/d